In [1]:
import requests
import pandas as pd
import time

def get_binance_futures_klines(symbol="ETHUSDT", interval="1m", limit=1000, days=5):
    """ ... """  # Keeping the docstring as is for clarity
    
    BASE_URL = "https://fapi.binance.com/fapi/v1/klines"
    
    end_time = int(time.time() * 1000)
    ms_in_a_day = 86400000
    total_ms = days * ms_in_a_day
    start_time = end_time - total_ms
    
    frames = []

    while end_time > start_time:
        params = {
            'symbol': symbol,
            'interval': interval,
            'limit': limit,
            'endTime': end_time
        }

        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        
        data = response.json()
        df = pd.DataFrame(data, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 
                                         'quote_asset_volume', 'number_of_trades', 'taker_buy_base', 
                                         'taker_buy_quote', 'ignore'])
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
        
        frames.insert(0, df)
        
        end_time = df.iloc[0]['open_time'].value

    final_df = pd.concat(frames, ignore_index=True)
    
    return final_df

df = get_binance_futures_klines()
df.to_csv('ETHUSDT_5days_min.csv', index=False)

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))